# Isochrone Precomputation: Oslo Bysykkel

Precomputes isochrone polygons for each station from trip data:
1. Per-connection (station pair) outlier filtering via IQR
2. Travel time matrix (median duration per pair; haversine estimate for missing)
3. Per-station interpolation onto grid + contour extraction
4. Export to prepared-data/isochrones.json

## Setup & Imports

In [1]:
# =============================================================================
# SETUP: Project paths and execution utils
# =============================================================================
from pathlib import Path
import json
import sys

cwd = Path.cwd()
project_root = cwd if (cwd / "package.json").exists() else cwd.parent.parent
raw_dir = project_root / "raw-data"
prepared_dir = project_root / "prepared-data"

sys.path.insert(0, str(project_root / "data-pipeline"))
from execution_utils import show_execution_banner, write_with_execution_metadata

print("Project root:", project_root)
print("Raw data:", raw_dir)
print("Prepared data:", prepared_dir)

out_path = prepared_dir / "isochrones.json"
_pipeline_start_time = show_execution_banner(out_path)

Project root: c:\Users\Nicol\Desktop\INF252-Course-Project
Raw data: c:\Users\Nicol\Desktop\INF252-Course-Project\raw-data
Prepared data: c:\Users\Nicol\Desktop\INF252-Course-Project\prepared-data
--- Last execution ---
  Timestamp: 2026-02-18T00:00:00
  Duration:  0s
  System:    ?
  CPUs:      ? | Processor: ?
  RAM:       N/A
------------------------


In [2]:
# =============================================================================
# IMPORTS
# =============================================================================
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

np.random.seed(42)

## Load Data

In [3]:
# =============================================================================
# Load trip data from raw-data/YYYY/MM.json
# =============================================================================
records = []
for year_dir in sorted(raw_dir.iterdir()):
    if not year_dir.is_dir():
        continue
    year = int(year_dir.name)
    for json_path in sorted(year_dir.glob("*.json")):
        month = int(json_path.stem)
        with open(json_path, encoding="utf-8") as f:
            data = json.load(f)
        trips = data if isinstance(data, list) else data.get("data", data.get("trips", []))
        for t in trips:
            records.append((year, month, t))

rows = []
for year, month, t in records:
    rows.append({
        "year": year,
        "month": month,
        "duration": t.get("duration"),
        "start_station_id": str(t.get("start_station_id")),
        "start_station_name": t.get("start_station_name"),
        "end_station_id": str(t.get("end_station_id")),
        "end_station_name": t.get("end_station_name"),
        "start_lat": t.get("start_station_latitude"),
        "start_lon": t.get("start_station_longitude"),
        "end_lat": t.get("end_station_latitude"),
        "end_lon": t.get("end_station_longitude"),
    })

df = pd.DataFrame(rows)
df = df.dropna(subset=["duration", "start_lat", "start_lon", "end_lat", "end_lon"])
df = df[df["start_station_id"] != df["end_station_id"]].copy()

print(f"Loaded {len(df)} trips (excluding same-station)")

Loaded 9761398 trips (excluding same-station)


## Phase 1: Per-Connection Outlier Filtering

In [4]:
# =============================================================================
# Per (start_station_id, end_station_id): IQR outlier removal
# =============================================================================
MIN_TRIPS_FOR_IQR = 5
IQR_MULTIPLIER = 1.5

def iqr_mask(g):
    if len(g) < MIN_TRIPS_FOR_IQR:
        return pd.Series(True, index=g.index)
    q1 = g.quantile(0.25)
    q3 = g.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        return pd.Series(True, index=g.index)
    low = q1 - IQR_MULTIPLIER * iqr
    high = q3 + IQR_MULTIPLIER * iqr
    return (g >= low) & (g <= high)

mask = df.groupby(["start_station_id", "end_station_id"])["duration"].transform(iqr_mask)
df_filtered = df[mask].copy()

print(f"After per-pair IQR filter: {len(df_filtered)} trips (from {len(df)})")

After per-pair IQR filter: 8863239 trips (from 9761398)


## Phase 2: Travel Time Matrix & Station Catalog

In [5]:
# =============================================================================
# Build station catalog (id, name, lat, lon) from first occurrence
# =============================================================================
stations_start = df_filtered.groupby("start_station_id").agg({
    "start_station_name": "first",
    "start_lat": "first",
    "start_lon": "first",
}).rename(columns={"start_station_name": "name", "start_lat": "lat", "start_lon": "lon"})
stations_end = df_filtered.groupby("end_station_id").agg({
    "end_station_name": "first",
    "end_lat": "first",
    "end_lon": "first",
}).rename(columns={"end_station_name": "name", "end_lat": "lat", "end_lon": "lon"})

all_ids = set(stations_start.index) | set(stations_end.index)
stations = []
for sid in sorted(all_ids, key=lambda x: (len(str(x)), x)):
    if sid in stations_start.index:
        row = stations_start.loc[sid]
    else:
        row = stations_end.loc[sid]
    stations.append({"id": sid, "name": str(row["name"]), "lat": float(row["lat"]), "lon": float(row["lon"])})

stations_df = pd.DataFrame(stations)
print(f"Stations: {len(stations)}")

Stations: 292


In [6]:
# =============================================================================
# Travel time matrix: median duration per (origin, dest) in seconds
# =============================================================================
tt_matrix = (
    df_filtered.groupby(["start_station_id", "end_station_id"])["duration"]
    .median()
    .reset_index()
)
tt_matrix.columns = ["origin", "dest", "duration_sec"]

# Median speed from filtered trips (haversine distance / duration)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

df_filtered["dist_km"] = haversine_km(
    df_filtered["start_lat"], df_filtered["start_lon"],
    df_filtered["end_lat"], df_filtered["end_lon"]
)
df_filtered["speed_km_per_sec"] = df_filtered["dist_km"] / df_filtered["duration"]
median_speed = df_filtered["speed_km_per_sec"].median()
print(f"Median speed (km/s): {median_speed:.6f}")

Median speed (km/s): 0.002725


In [7]:
# =============================================================================
# For each origin station: get travel times to all dests (observed or estimated)
# =============================================================================
def get_travel_times_from_origin(origin_id):
    origin_row = stations_df[stations_df["id"] == origin_id].iloc[0]
    o_lat, o_lon = origin_row["lat"], origin_row["lon"]
    results = []
    for _, dest_row in stations_df.iterrows():
        if dest_row["id"] == origin_id:
            results.append((dest_row["lat"], dest_row["lon"], 0.0))
            continue
        match = tt_matrix[(tt_matrix["origin"] == origin_id) & (tt_matrix["dest"] == dest_row["id"])]
        if len(match) > 0:
            dur_min = match.iloc[0]["duration_sec"] / 60.0
        else:
            d_km = haversine_km(o_lat, o_lon, dest_row["lat"], dest_row["lon"])
            dur_min = (d_km / median_speed) / 60.0
        results.append((dest_row["lat"], dest_row["lon"], dur_min))
    return results

## Phase 3: Interpolate & Extract Contours

In [8]:
# =============================================================================
# Grid bounds (Oslo) and resolution
# =============================================================================
LAT_MIN = 59.898
LAT_MAX = 59.955
LON_MIN = 10.648
LON_MAX = 10.818
GRID_N = 100
TIME_BANDS_MIN = [5, 10, 15, 20]

In [9]:
# =============================================================================
# Contour to GeoJSON polygon (manual extraction from matplotlib)
# =============================================================================
def contour_to_geojson_polygon(lon_2d, lat_2d, z, level):
    """Extract contour at level, return GeoJSON Polygon or None."""
    fig, ax = plt.subplots()
    cs = ax.contour(lon_2d, lat_2d, z, levels=[level])
    plt.close(fig)
    paths = cs.get_paths()
    if len(paths) == 0:
        return None
    if len(paths) == 0:
        return None
    # Take largest path (main boundary)
    path = max(paths, key=lambda p: len(p.vertices))
    verts = path.vertices
    coords = [[float(x), float(y)] for x, y in verts]
    if coords and (coords[0][0] != coords[-1][0] or coords[0][1] != coords[-1][1]):
        coords.append(coords[0])
    return {"type": "Polygon", "coordinates": [coords]}

def compute_isochrones_for_station(origin_id):
    points = get_travel_times_from_origin(origin_id)
    lats = np.array([p[0] for p in points])
    lons = np.array([p[1] for p in points])
    values = np.array([p[2] for p in points])
    lon_1d = np.linspace(LON_MIN, LON_MAX, GRID_N)
    lat_1d = np.linspace(LAT_MIN, LAT_MAX, GRID_N)
    lon_2d, lat_2d = np.meshgrid(lon_1d, lat_1d)
    zi = griddata((lons, lats), values, (lon_2d, lat_2d), method="linear", fill_value=60.0)
    polygons = {}
    for t in TIME_BANDS_MIN:
        poly = contour_to_geojson_polygon(lon_2d, lat_2d, zi, t)
        if poly is not None:
            polygons[str(t)] = poly
    return polygons

In [10]:
# =============================================================================
# Compute isochrones for all stations
# =============================================================================
isochrones = {}
for i, row in enumerate(stations_df.itertuples()):
    if (i + 1) % 50 == 0 or i == 0:
        print(f"Processing station {i+1}/{len(stations_df)}: {row.name}")
    isochrones[row.id] = compute_isochrones_for_station(row.id)

print(f"Done. Computed isochrones for {len(isochrones)} stations.")

Processing station 1/292: Tøyenparken
Processing station 50/292: Briskeby
Processing station 100/292: Jernbanetorget
Processing station 150/292: Bryn T-Bane
Processing station 200/292: Henrik Wergelands allé
Processing station 250/292: Bygdøy allé
Done. Computed isochrones for 292 stations.


## Export

In [11]:
# =============================================================================
# Export to prepared-data/isochrones.json
# =============================================================================
export_data = {
    "stations": stations,
    "time_bands_min": TIME_BANDS_MIN,
    "isochrones": isochrones,
}

write_with_execution_metadata(out_path, export_data, _pipeline_start_time)
print(f"Wrote {out_path}")

Wrote c:\Users\Nicol\Desktop\INF252-Course-Project\prepared-data\isochrones.json
